In [18]:
## IMPORTS
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import recordlinkage

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [34]:
h2a = pd.read_excel("./my_data/h2a_2018.xlsx")
h2a.head()

url = "https://enfxfr.dol.gov/data_catalog/WHD/whd_whisard_20210415.csv.zip"
raw_dol = pd.read_csv(url, 
                      index_col=None,
                      dtype={7:'string'})
raw_dol['findings_start_date'] = pd.to_datetime(raw_dol['findings_start_date'], errors='coerce')
raw_dol['findings_end_date'] = pd.to_datetime(raw_dol['findings_end_date'], errors='coerce')
print(f'raw dataframe has {len(raw_dol.columns)} columns and {len(raw_dol)} rows')
raw_dol.head(5)

,CASE_NO,DECISION_DATE,VISA_CLASS,CASE_RECEIVED_DATE,CASE_STATUS,REQUESTED_START_DATE_OF_NEED,REQUESTED_END_DATE_OF_NEED,PRIMARY_SUB,EMPLOYER_NAME,TRADE_NAME_DBA,...,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,OTHER_WORKSITE_LOCATION,ORGANIZATION_FLAG,SWA_NAME,JOB_IDNUMBER,JOB_START_DATE,JOB_END_DATE
0,H-300-17263-482969,2017-10-02 12:46:58,H-2A,2017-09-20,DETERMINATION ISSUED - WITHDRAWN,2017-11-05,2018-05-31,PRI,"Gulf Harvesting, Inc.",NaN,...,Felda,Hendry,FL,33930,Y,H-2A Labor Contractor or Job Contractor,Florida Department of Economic Opportunity,NaN,NaT,NaT
1,H-300-17258-492669,2017-10-02 13:46:39,H-2A,2017-09-15,DETERMINATION ISSUED - CERTIFICATION,2017-12-01,2018-11-29,PRI,ECOSYSTEM CONCEPTS INC.,NaN,...,Dixon,Solano,CA,95620,Y,Association - Filing as Agent (H-2A Only),California Employment Development Department,15575066,2017-09-26,2018-05-31
2,H-300-17240-814010,2017-10-02 15:40:18,H-2A,2017-08-28,DETERMINATION ISSUED - CERTIFICATION EXPIRED,2017-11-01,2018-07-31,PRI,FISHER BROTHERS CATTLE,NaN,...,Anderson,Grimes,TX,77830,Y,Individual Employer,Texas Workforce Commission,TX6572948,2017-08-23,2018-03-17
3,H-300-17244-028701,2017-10-02 15:52:12,H-2A,2017-09-01,DETERMINATION ISSUED - CERTIFICATION EXPIRED,2017-11-01,2018-06-15,PRI,"CIRCLE H CITRUS, INC.",NaN,...,Arcadia,DeSoto,FL,34266,Y,H-2A Labor Contractor or Job Contractor,Department of Economic Opportunity,FL10543312,2017-08-25,2018-02-22
4,H-300-17248-540912,2017-10-02 15:54:55,H-2A,2017-09-05,DETERMINATION ISSUED - PARTIAL CERTIFICATION E...,2017-10-20,2018-03-17,PRI,"FRESH HARVEST, INC.",NaN,...,Mecca,Riverside,CA,92274,Y,H-2A Labor Contractor or Job Contractor,California EDD,15530682,2017-08-31,2018-01-02


raw dataframe has 110 columns and 313928 rows


,case_id,trade_nm,legal_name,street_addr_1_txt,cty_nm,st_cd,zip_cd,naic_cd,naics_code_description,case_violtn_cnt,...,flsa_smwsl_bw_atp_amt,flsa_smwsl_ee_atp_cnt,eev_violtn_cnt,h2b_violtn_cnt,h2b_bw_atp_amt,h2b_ee_atp_cnt,sraw_violtn_cnt,sraw_bw_atp_amt,sraw_ee_atp_cnt,ld_dt
0,1428484,Reliant Energy,"Reliant Energy Retail Services, LLC",1000 Main,Houston,TX,77002.0,09310,State Generation and Distribution of Electric ...,0,...,0.0,0,0,0,0.0,0,0,0.0,0,2015-02-20 01:00:06 EST
1,1784005,Healthcare Services Group at Westwood Center,"Healthcare Services Group, Inc.",298 Main Street,Keene,NH,3431.0,0967,Local Nursing Homes,1,...,0.0,0,0,0,0.0,0,0,0.0,0,2016-06-11 02:22:25 EDT
2,1460300,Central Avenue Bakery,"Central Avenue Bakery, Inc.",679 Central Avenue,Pawtucket,RI,2861.0,311811,Retail Bakeries,4,...,0.0,0,0,0,0.0,0,0,0.0,0,2015-02-20 01:00:06 EST
3,1294821,Gallagher Services,Catholic Cherities,2520 Pot Spring Road,Lutherville Timonium,MD,21093.0,09690,Local Group Homes for Mentally and Physically ...,0,...,0.0,0,0,0,0.0,0,0,0.0,0,2015-02-20 01:00:06 EST
4,1437099,Dragon Dynasty Oriental Mart,NaN,2299 South Ridgewood Avenue,Daytona Beach,FL,32119.0,42449,Other Grocery and Related Products Merchant Wh...,0,...,0.0,0,0,0,0.0,0,0,0.0,0,2015-02-20 01:00:06 EST


In [37]:
def find_status(one):
    string_version = str(one)
    pattern = r'\-\s(.*)$'
    found = re.findall(pattern, string_version)
    return found[0]
h2a["status"] = [find_status(one) for one in h2a.CASE_STATUS]
h2a.status.value_counts()
approved_only = h2a.loc[((h2a.status == "CERTIFICATION") | (h2a.status == "PARTIAL CERTIFICATION")),:].copy()
approved_only.head()
approved_only.shape
approved_only.EMPLOYER_NAME.value_counts()

CERTIFICATION                    11792
CERTIFICATION EXPIRED             1994
PARTIAL CERTIFICATION              235
WITHDRAWN                          221
DENIED                             165
PARTIAL CERTIFICATION EXPIRED       45
Name: status, dtype: int64

,CASE_NO,DECISION_DATE,VISA_CLASS,CASE_RECEIVED_DATE,CASE_STATUS,REQUESTED_START_DATE_OF_NEED,REQUESTED_END_DATE_OF_NEED,PRIMARY_SUB,EMPLOYER_NAME,TRADE_NAME_DBA,...,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,OTHER_WORKSITE_LOCATION,ORGANIZATION_FLAG,SWA_NAME,JOB_IDNUMBER,JOB_START_DATE,JOB_END_DATE,status
1,H-300-17258-492669,2017-10-02 13:46:39,H-2A,2017-09-15,DETERMINATION ISSUED - CERTIFICATION,2017-12-01,2018-11-29,PRI,ECOSYSTEM CONCEPTS INC.,NaN,...,Solano,CA,95620,Y,Association - Filing as Agent (H-2A Only),California Employment Development Department,15575066,2017-09-26,2018-05-31,CERTIFICATION
11,H-300-17257-446860,2017-10-02 16:36:19,H-2A,2017-09-14,DETERMINATION ISSUED - CERTIFICATION,2017-12-08,2018-12-06,PRI,WESTERN RANGE ASSOCIATION,NaN,...,FRESNO,CA,93210,Y,Association - Joint Employer (H-2A Only),California Employment Development Department,15567271,2017-09-20,2018-06-07,CERTIFICATION
12,H-300-17257-446860,2017-10-02 16:36:19,H-2A,2017-09-14,DETERMINATION ISSUED - CERTIFICATION,2017-12-08,2018-12-06,SUB,SHEEP COMPANY LLC,NaN,...,FRESNO,CA,93210,Y,NaN,California Employment Development Department,15567271,2017-09-20,2018-06-07,CERTIFICATION
13,H-300-17262-816860,2017-10-02 16:44:54,H-2A,2017-09-19,DETERMINATION ISSUED - CERTIFICATION,2018-01-01,2018-12-30,PRI,Sieben Ranch Co.,NaN,...,Lewis & Clark,MT,59648,Y,Association - Filing as Agent (H-2A Only),Montana State Workforce,10316116,2017-09-21,2018-06-29,CERTIFICATION
19,H-300-17256-339375,2017-10-03 13:28:16,H-2A,2017-09-13,DETERMINATION ISSUED - CERTIFICATION,2017-12-08,2018-12-06,PRI,WESTERN RANGE ASSOCIATION,NaN,...,MOFFAT,CO,81625,Y,Association - Joint Employer (H-2A Only),Colorado Department of Labor & Employment,CO 7136889,2017-09-15,2018-06-08,CERTIFICATION


(12027, 63)

12027